# MS2 Feature Evaluation

Comprehensive evaluation of MS2 features on full dataset with target-decoy analysis.


In [ ]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from alpharaw import register_all_readers
from alphabase.peptide.fragment import get_charged_frag_types
from peptdeep.rescore.fdr import calc_fdr

from dia_aspire_rescore.io import read_diann2
from dia_aspire_rescore.psm.matcher import DIAPeptideSpectrumMatcher
from dia_aspire_rescore.config import FineTuneConfig
from dia_aspire_rescore.finetuning import FineTuner
from dia_aspire_rescore.features import MS2FeatureGenerator
from dia_aspire_rescore.plot import plot_target_decoy_dist, plot_qvalues

register_all_readers()

# Create output directory
output_dir = Path('./output/ms2')
output_dir.mkdir(parents=True, exist_ok=True)


/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.11/site-packages/alpharaw/raw_access/pysciexwifffilereader.py:44: UserWarning: Dotnet-based dependencies could not be loaded. Sciex support is disabled.
  warnings.warn(


In [ ]:
psm_df_all = read_diann2("../data/raw/SYS026_RA957/DDA_SYSMHC_bynam/lib-base-result-first-pass.parquet")
psm_df_all = psm_df_all.sort_values(by='nAA', ascending=True).reset_index(drop=True)
print(f"Loaded {len(psm_df_all)} PSMs")
print(f"Target: {(psm_df_all['decoy']==0).sum()}, Decoy: {(psm_df_all['decoy']==1).sum()}")


/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.11/site-packages/alphabase/psm_reader/psm_reader.py:342: UserWarning: Unknown modifications: {'Q(UniMod:1)'}. Precursors with unknown modifications will be removed.
  warnings.warn(


Loaded 338839 PSMs
Target: 174407, Decoy: 164432


In [ ]:

# Select high-confidence PSMs for finetuning
psm_df_finetune = psm_df_all[
    (psm_df_all['fdr1_search1'] < 0.01) & 
    (psm_df_all['decoy'] == 0)
].copy()
print(f"Selected {len(psm_df_finetune)} high-confidence PSMs for finetuning")


Selected 82364 high-confidence PSMs for finetuning


In [4]:

# Cell 4: Match MS2 for finetuning data
ms_files = {
    '20200317_QE_HFX2_LC3_DIA_RA957_R01': './output/20200317_QE_HFX2_LC3_DIA_RA957_R01.mzML.hdf5',
    '20200317_QE_HFX2_LC3_DIA_RA957_R02': './output/20200317_QE_HFX2_LC3_DIA_RA957_R02.mzML.hdf5'
}

matcher = DIAPeptideSpectrumMatcher(n_neighbors=0)
psm_df_finetune, _, matched_intensity_finetune, _ = matcher.match_ms2_multi_raw(
    psm_df_finetune, ms_files, 'hdf5'
)
print(f"Matched {len(psm_df_finetune)} PSMs for finetuning")


100%|██████████| 2/2 [00:05<00:00,  2.70s/it]

Matched 82364 PSMs for finetuning


In [5]:

# Finetune MS2 model
config = FineTuneConfig(
    instrument='QE',
    nce=27,
    psm_num_to_train_ms2=min(5000, len(psm_df_finetune)),
    epoch_to_train_ms2=15,
    train_verbose=True
)

finetuner = FineTuner(config)
finetuner.load_pretrained('generic')
finetuner.train_ms2(psm_df_finetune, matched_intensity_finetune)
print("MS2 model finetuned successfully")


/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.11/site-packages/peptdeep/model/ms2.py:416: UserWarning: mask_modloss is deprecated and will be removed in the future. To mask the modloss fragments, the charged_frag_types should not include the modloss fragments.
  warnings.warn(


2025-12-06 03:03:55> 5135 PSMs for MS2 model training/transfer learning
2025-12-06 03:03:55> Training with fixed sequence length: 0
[Training] Epoch=1, lr=2e-05, loss=0.07773217558860779
[Training] Epoch=2, lr=3e-05, loss=0.07299942125876745
[Training] Epoch=3, lr=4e-05, loss=0.06392019887765249
[Training] Epoch=4, lr=5e-05, loss=0.06565267791350683
[Training] Epoch=5, lr=6e-05, loss=0.05767254289239645
[Training] Epoch=6, lr=7e-05, loss=0.059036475916703544
[Training] Epoch=7, lr=8e-05, loss=0.05679500003655751
[Training] Epoch=8, lr=9e-05, loss=0.055041649068395294
[Training] Epoch=9, lr=0.0001, loss=0.057756325354178746
[Training] Epoch=10, lr=0.0001, loss=0.05625560035308202
[Training] Epoch=11, lr=9.045084971874738e-05, loss=0.051625431329011914
[Training] Epoch=12, lr=6.545084971874738e-05, loss=0.05023529467483361
[Training] Epoch=13, lr=3.4549150281252636e-05, loss=0.05427727947632472
[Training] Epoch=14, lr=9.549150281252633e-06, loss=0.050245413184165956
[Training] Epoch=15, 

In [7]:
# Match MS2 for all data
psm_df_all, fragment_mz_df, matched_intensity_df, matched_mz_err_df = matcher.match_ms2_multi_raw(
    psm_df_all, ms_files, 'hdf5'
)
print(f"Matched {len(psm_df_all)} PSMs for feature generation")


100%|██████████| 2/2 [00:04<00:00,  2.29s/it]

Matched 338839 PSMs for feature generation


In [8]:
# Generate MS2 features
frag_types = get_charged_frag_types(['b', 'y'], 2)
ms2_generator = MS2FeatureGenerator(
    model_mgr=finetuner.model_manager,
    frag_types=frag_types,
    spc_top_k=10
)

psm_df_all = ms2_generator.generate(psm_df_all, matched_intensity_df, matched_mz_err_df)
print(f"Generated {len(ms2_generator.feature_names)} MS2 features")


2025-12-06 03:05:13> Predicting MS2 ...


100%|██████████| 7/7 [01:28<00:00, 12.58s/it]


Generated 51 MS2 features


In [11]:
psm_df_all

,raw_name,sequence,charge,rt,rt_start,rt_stop,mobility,proteins,uniprot_ids,genes,...,matched_yion_ratio,both_matched_pred_yion_num,both_matched_pred_yion_to_matched,both_matched_pred_yion_to_pred,matched_not_pred_yion_num,matched_not_pred_yion_ratio,pred_not_matched_yion_num,pred_not_matched_yion_ratio,matched_yion_rel_to_pred,pred_yion_rel_to_matched
0,20200317_QE_HFX2_LC3_DIA_RA957_R01,YQYYHRYY,3,44.166344,44.104832,44.227886,0.0,1/sp|O96000|NDUBA_HUMAN,1/sp|O96000|NDUBA_HUMAN,,...,0.142857,0.0,0.0,0,2.0,1.0,0.0,0,0.0,0.0
1,20200317_QE_HFX2_LC3_DIA_RA957_R01,ERMGANSL,2,38.052963,37.930332,38.206379,0.0,1/sp|P52272|HNRPM_HUMAN,1/sp|P52272|HNRPM_HUMAN,,...,0.142857,0.0,0.0,0,2.0,1.0,0.0,0,0.0,0.0
2,20200317_QE_HFX2_LC3_DIA_RA957_R01,LPGPGASL,1,73.935051,73.812119,74.120216,0.0,1/sp|Q9Y6J0|CABIN_HUMAN,1/sp|Q9Y6J0|CABIN_HUMAN,,...,0.357143,0.0,0.0,0,5.0,1.0,0.0,0,0.0,0.0
3,20200317_QE_HFX2_LC3_DIA_RA957_R01,LPGPGAEL,1,79.160156,79.067657,79.252640,0.0,2/sp|Q5U651|RAIN_HUMAN/tr|M0R148|M0R148_HUMAN,2/sp|Q5U651|RAIN_HUMAN/tr|M0R148|M0R148_HUMAN,,...,0.214286,0.0,0.0,0,3.0,1.0,0.0,0,0.0,0.0
4,20200317_QE_HFX2_LC3_DIA_RA957_R01,LPGPAESL,1,75.566612,75.290459,75.842407,0.0,1/sp|Q86WR7|PRSR2_HUMAN,1/sp|Q86WR7|PRSR2_HUMAN,,...,0.428571,0.0,0.0,0,6.0,1.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338834,20200317_QE_HFX2_LC3_DIA_RA957_R02,DTFRPDLSSASFSR,2,73.139771,72.863914,73.262627,0.0,1/sp|Q13077|TRAF1_HUMAN,1/sp|Q13077|TRAF1_HUMAN,,...,0.076923,0.0,0.0,0,2.0,1.0,0.0,0,0.0,0.0
338835,20200317_QE_HFX2_LC3_DIA_RA957_R02,SFHDPAGALQAAVR,2,75.013680,74.890953,75.197929,0.0,1/sp|O15533|TPSN_HUMAN,1/sp|O15533|TPSN_HUMAN,,...,0.076923,0.0,0.0,0,2.0,1.0,0.0,0,0.0,0.0
338836,20200317_QE_HFX2_LC3_DIA_RA957_R02,SSFPSASLGKASVR,2,58.953045,58.828590,59.046440,0.0,1/sp|Q9NWN3|FBX34_HUMAN,1/sp|Q9NWN3|FBX34_HUMAN,,...,0.153846,0.0,0.0,0,4.0,1.0,0.0,0,0.0,0.0
338837,20200317_QE_HFX2_LC3_DIA_RA957_R02,DAFNHLTTWLEDTR,3,107.905006,107.873901,108.122513,0.0,3/tr|H7C125|H7C125_HUMAN/tr|E9PKL7|E9PKL7_HUMA...,3/tr|H7C125|H7C125_HUMAN/tr|E9PKL7|E9PKL7_HUMA...,,...,0.038462,0.0,0.0,0,1.0,1.0,0.0,0,0.0,0.0


In [12]:
# Cell 8: Evaluate each feature (修改版)
for feature in ms2_generator.feature_names:
    print(f"Evaluating {feature}...")
    
    # Check if feature has variation
    feature_values = psm_df_all[feature].dropna()
    if feature_values.std() < 1e-10:
        print(f"  Skipping {feature} - no variation in values")
        continue
    
    # Calculate FDR using this feature as score
    try:
        psm_df_eval = calc_fdr(psm_df_all.copy(), feature, ascending=False)
    except Exception as e:
        print(f"  Error calculating FDR for {feature}: {e}")
        continue
    
    # Create 2-column figure
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Left: Target-Decoy distribution
    try:
        # Disable KDE if data has low variation
        use_kde = feature_values.std() > 0.01
        plot_target_decoy_dist(psm_df_eval, feature, ax=axes[0], kde=use_kde)
        axes[0].set_title(f'{feature} - Target/Decoy Distribution')
    except Exception as e:
        print(f"  Error plotting distribution for {feature}: {e}")
        axes[0].text(0.5, 0.5, f'Error: {str(e)[:50]}', 
                     ha='center', va='center', transform=axes[0].transAxes)
        axes[0].set_title(f'{feature} - Error in Distribution Plot')
    
    # Right: Q-value curve
    try:
        target_psms = psm_df_eval[psm_df_eval['decoy'] == 0]
        if len(target_psms) > 0:
            plot_qvalues(target_psms['fdr'].values, threshold=0.1, ax=axes[1])
            axes[1].set_title(f'{feature} - Discoveries at FDR')
        else:
            axes[1].text(0.5, 0.5, 'No target PSMs', 
                        ha='center', va='center', transform=axes[1].transAxes)
    except Exception as e:
        print(f"  Error plotting q-values for {feature}: {e}")
        axes[1].text(0.5, 0.5, f'Error: {str(e)[:50]}', 
                     ha='center', va='center', transform=axes[1].transAxes)
        axes[1].set_title(f'{feature} - Error in Q-value Plot')
    
    plt.tight_layout()
    
    # Save to PDF
    pdf_path = output_dir / f'{feature}_evaluation.pdf'
    try:
        plt.savefig(pdf_path, bbox_inches='tight', dpi=300)
        print(f"  Saved to {pdf_path}")
    except Exception as e:
        print(f"  Error saving PDF for {feature}: {e}")
    finally:
        plt.close()

print(f"\nAll evaluations saved to {output_dir}")

Evaluating cos...
  Skipping cos - no variation in values
Evaluating sa...
  Skipping sa - no variation in values
Evaluating spc...
  Error calculating FDR for spc: calc_fdr() got an unexpected keyword argument 'ascending'
Evaluating pcc...
  Skipping pcc - no variation in values
Evaluating cos_bion...
  Skipping cos_bion - no variation in values
Evaluating sa_bion...
  Skipping sa_bion - no variation in values
Evaluating spc_bion...
  Error calculating FDR for spc_bion: calc_fdr() got an unexpected keyword argument 'ascending'
Evaluating pcc_bion...
  Skipping pcc_bion - no variation in values
Evaluating cos_yion...
  Skipping cos_yion - no variation in values
Evaluating sa_yion...
  Skipping sa_yion - no variation in values
Evaluating spc_yion...
  Error calculating FDR for spc_yion: calc_fdr() got an unexpected keyword argument 'ascending'
Evaluating pcc_yion...
  Skipping pcc_yion - no variation in values
Evaluating merr_weighted_frag_score...
  Error calculating FDR for merr_weigh

In [14]:

# Cell 9: Summary statistics
summary = []
for feature in ms2_generator.feature_names:
    psm_df_eval = calc_fdr(psm_df_all.copy(), feature)
    targets_01fdr = ((psm_df_eval['fdr'] <= 0.01) & (psm_df_eval['decoy'] == 0)).sum()
    summary.append({
        'feature': feature,
        'targets_at_1pct_fdr': targets_01fdr
    })

summary_df = pd.DataFrame(summary).sort_values('targets_at_1pct_fdr', ascending=False)
print("\nFeature Performance Summary (Targets at 1% FDR):")
print(summary_df)

# Save summary
summary_df.to_csv(output_dir / 'feature_summary.csv', index=False)

/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.11/site-packages/peptdeep/rescore/fdr.py:61: RuntimeWarning: divide by zero encountered in divide
  fdr_values = decoy_cumsum / target_cumsum
/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.11/site-packages/peptdeep/rescore/fdr.py:61: RuntimeWarning: divide by zero encountered in divide
  fdr_values = decoy_cumsum / target_cumsum
/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.11/site-packages/peptdeep/rescore/fdr.py:61: RuntimeWarning: divide by zero encountered in divide
  fdr_values = decoy_cumsum / target_cumsum
/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.11/site-packages/peptdeep/rescore/fdr.py:61: RuntimeWarning: divide by zero encountered in divide
  fdr_values = decoy_cumsum / target_cumsum
/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.11/site-packages/peptdeep/rescore/fdr.py:61: RuntimeWarning: divide by zero encountered in divide
  fdr_values = decoy_cumsum / targ


Feature Performance Summary (Targets at 1% FDR):
                              feature  targets_at_1pct_fdr
12           merr_weighted_frag_score                28524
16           merr_weighted_yion_score                24930
19                 matched_frag_ratio                21091
41                 matched_yion_ratio                20119
23          matched_not_pred_frag_num                19693
18                   matched_frag_num                19693
45          matched_not_pred_yion_num                17772
40                   matched_yion_num                17772
14           merr_weighted_bion_score                13637
30                 matched_bion_ratio                11259
29                   matched_bion_num                 7231
34          matched_not_pred_bion_num                 7231
42         both_matched_pred_yion_num                    0
36          pred_not_matched_bion_num                    0
31         both_matched_pred_bion_num                    0
32  bo

/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.11/site-packages/peptdeep/rescore/fdr.py:61: RuntimeWarning: divide by zero encountered in divide
  fdr_values = decoy_cumsum / target_cumsum
